# Estimate area change required for glaciers to be in balance with climate

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns

data_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'

In [ ]:
# Load AOIs
aois_fn = os.path.join(data_path, 'compiled_data', 'all_aois_climate_cluster.shp')
aois = gpd.read_file(aois_fn)
aois['TermType'] = aois['TermType'].astype(int)
aois

In [ ]:
# Load median AARs
aars_fn = os.path.join(data_path, 'results', 'min_snow_cover_stats.csv')
aars = pd.read_csv(aars_fn)
aars

## Estimate balanced AARs and areas based on terminus type

In [ ]:
# Function for determining terminus type and balanced AARs
# See RGI v. 6 documentation, pg. 16: https://nsidc.org/sites/default/files/rgi_tech_report_v60.pdf
def determine_terminus_type_balanced_aar(term_type_int):
    if term_type_int==0:
        term_type_str = 'Land-terminating'
        balance_aar = 0.5
    elif term_type_int==1:
        term_type_str = 'Marine-terminating'
        balance_aar = 0.9
    elif term_type_int==2:
        term_type_str = 'Lake-terminating'
        balance_aar = 0.9
    return term_type_str, balance_aar

# Function for determining balance area
def balance_area(A_glacier, aar, balance_aar):
    # A_balanced = A_accum / balance_aar
    # A_accum = A_glacier * aar

    # calculate accumulation area
    A_accum = A_glacier * aar
    # calculate balanced area
    A_balanced = A_accum / balance_aar
    # calculate change in area
    dA = A_glacier - A_balanced

    return A_balanced, dA
    

In [ ]:
# Initialize a dataframe for storing results
balance_areas = pd.DataFrame()

# Iterate over sites
for i in range(0,len(aois)):
    # grab ID, area, and terminus type from AOI
    glacier_id = aois.loc[i, 'RGIId']
    area = aois.loc[i, 'Area']
    term_type_int = aois.loc[i, 'TermType']
    
    # grab median AAR
    aar = aars.loc[aars['RGIId']==glacier_id, 'AAR_P50_min'].values[0]

    # estimate balanced aar based on terminus type
    term_type_str, aar_balance = determine_terminus_type_balanced_aar(term_type_int)

    # calculate balanced area and change in area
    area_balanced, dA = balance_area(area, aar, aar_balance)

    # compile in dataframe
    df = pd.DataFrame({'RGIId': [glacier_id], 
                       'Terminus type [int]': [term_type_int],
                       'Terminus type [str]': [term_type_str],
                       'AAR': [aar],
                       'Area [km2]': [area],
                       'Balance AAR': [aar_balance],
                       'Balance area [km2]': [area_balanced],
                       'dArea [km2]': [dA]
                       })
    balance_areas = pd.concat([balance_areas, df])

# Adjust dataframe
balance_areas.sort_values(by='RGIId', inplace=True)
balance_areas.reset_index(drop=True, inplace=True)

# Save to file
balance_areas_fn = os.path.join(data_path, 'results', 'balanced_areas.csv')
balance_areas.to_csv(balance_areas_fn, index=False)
print('Balanced areas saved to file:', balance_areas_fn)

balance_areas

In [ ]:
# Plot some results
balance_areas_fn = os.path.join(data_path, 'results', 'balanced_areas.csv')
balance_areas = pd.read_csv(balance_areas_fn)

fig, ax = plt.subplots()
sns.histplot(data=balance_areas, x='dArea [km2]', hue='Terminus type [str]', ax=ax)
ax.set_xlabel('$\Delta$Area [km$^2$]')
ax.set_ylabel('Counts')
plt.show()